In [ ]:
import yfinance as yf
import pandas as pd

def calculate_RSI(ticker, period='1mo', interval='1d', window_length=14):
    """
    Calculate the Relative Strength Index (RSI) for a given stock ticker.

    :param ticker: Stock ticker symbol
    :param period: Period to fetch data for (default '1mo' - 1 month)
    :param interval: Data interval (default '1d' - daily)
    :param window_length: Window length for RSI calculation (default 14)
    :return: RSI value for the last day in the fetched data
    """
    # Fetch historical data from Yahoo Finance
    data = yf.download(ticker, period=period, interval=interval)
    
    # Ensure data is fetched successfully
    if data.empty:
        raise ValueError(f"No data found for ticker {ticker}")

    # Calculate the price changes
    delta = data['Close'].diff()

    # Separate gains and losses
    gain = (delta.where(delta > 0, 0)).fillna(0)
    loss = (-delta.where(delta < 0, 0)).fillna(0)

    # Calculate the average gain and average loss
    avg_gain = gain.rolling(window=window_length, min_periods=1).mean()
    avg_loss = loss.rolling(window=window_length, min_periods=1).mean()

    # Calculate the Relative Strength (RS)
    rs = avg_gain / avg_loss

    # Calculate the RSI
    rsi = 100 - (100 / (1 + rs))

    # Get the last RSI value
    last_rsi = rsi.iloc[-1]

    return last_rsi

In [ ]:
def calculate_SMA(ticker, window_length, period='6mo', interval='1d'):
    """
    Calculate the Simple Moving Average (SMA) for a given stock ticker and window length.

    :param ticker: Stock ticker symbol
    :param window_length: Window length for SMA calculation (e.g., 20 or 50)
    :param period: Period to fetch data for (default '6mo' - 6 months)
    :param interval: Data interval (default '1d' - daily)
    :return: SMA value for the last day in the fetched data
    """
    # Fetch historical data from Yahoo Finance
    data = yf.download(ticker, period=period, interval=interval)
    
    # Ensure data is fetched successfully
    if data.empty:
        raise ValueError(f"No data found for ticker {ticker}")

    # Calculate the moving average
    sma = data['Close'].rolling(window=window_length).mean()
    
    # Get the last moving average value
    last_sma = sma.iloc[-1]
    
    return last_sma


In [ ]:
def Status(row):    
    if row["RSI"] > 70 and row["SMA_20"] > row["SMA_50"]: 
        return 1.0
    elif row["RSI"] < 30 and row["SMA_20"] < row["SMA_50"]: 
        return -1.0
    else:
        return 0.0
              

In [ ]:
# List of stock tickers
tickers = ['ADCINDIA.BO','ALLCARGO.BO','ASHIMASYN.BO','BSESN','FORBESCO.BO','GTNINDS.BO','GLOBOFFS.BO','GOODRICKE.BO','HOCL.BO',
          'ITHL.BO','KENNAMET.BO','KINETICENG.BO','KIRLFER.BO','PATSPINLTD.BO','PUNJCOMMU.BO','WARRENTEA.BO','20MICRONS.NS',
          '360ONE.NS','3IINFOLTD.NS','3MINDIA.BO','63MOONS.NS','ABB.NS','ACC.NS','AGI.NS','AIAENG.NS','AJRINFRA.NS',
           'AMJLAND.NS','APLAPOLLO.NS','ARSSINFRA.NS','AUBANK.NS','AARTIDRUGS.NS','AARTIIND.NS','AAVAS.BO','ABAN.NS',
          'ABBOTINDIA.NS','ACCELYA.NS','ADANIENSOL.NS','ADANIENT.NS','ADANIGREEN.NS','ADANIPORTS.NS','ADANIPOWER.NS','ATGL.NS',
          'AWL.NS','ABCAPITAL.NS','ABFRL.NS','ABSLAMC.NS','ADORWELD.NS','ADVENZYMES.NS','AEGISCHEM.NS','AETHER.NS','AFFLE.NS',
          'ATFL.NS','AHLUCONT.NS','AJANTPHARM.NS','AJMERA.NS','AKSHOPTFBR.NS','AKZOINDIA.NS','ALEMBICLTD.NS','APLLTD.NS',
           'ALKEM.NS','ALKYLAMINE.NS','ADSL.NS','ALOKINDS.NS','ARE.M.NS','AMBER.NS','AMBUJACEM.NS','ANANTRAJ.NS',
           'ANDHRAPAP.NS','ANDHRSUGAR.NS','ANGELONE.NS','ANSALAPI.NS','ANURAS.NS','APARINDS.NS','APOLLOHOSP.NS',
           'APOLLOTYRE.NS','APTECHT.NS','APTUS.NS','ACI.NS','ARCHIES.NS','ARSHIYA.NS','ARVINDFASN.NS','ARVIND.NS',
           'ASAHIINDIA.NS','ASHIANA.NS','ASHOKLEY.NS','ASHOKA.NS','ASIANHOTNR.NS','ASIANPAINT.NS','ASTERDM.NS','ASTRAL.NS',
          'ASTRAZEN.NS','ATLASCYCLE.NS','ATUL.NS','AUROPHARMA.NS','AUTOIND.NS','AUTOAXLES.NS','AVANTIFEED.NS','DMART.NS',
          'AXISBANK.NS','AXISCADES.NS','BASF.NS','BEML.NS','BFUTILITIE.NS','BGRENERGY.NS','BLKASHYAP.NS','INDIAGLYCO.NS','BPL.NS','BSL.NS',
          'BAJAJ-AUTO.BO','BAJAJCON.NS','BAJAJELEC.NS','BAJFINANCE.NS','BAJAJFINSV.NS','BAJAJHIND.NS','BAJAJHLDNG.NS',
          'BALAMINES.NS','BALAJITELE.NS','BALKRISIND.NS','BALLARPUR.NS','BALMLAWRIE.NS','BALRAMCHIN.NS','BANCOINDIA.NS',
          'BANDHANBNK.NS','BANKBARODA.NS','BANKINDIA.NS','MAHABANK.NS','BANARISUG.NS','BATAINDIA.NS','BAYERCROP.NS',
          'BERGEPAINT.NS','BEPL.NS','BDL.NS','BEL.NS','BHARATFORG.NS','BHEL.NS','BPCL.NS','BHARATRAS.NS','BHARTIARTL.NS',
           'BIKAJI.NS','BINANIIND.NS','BIOCON.NS','BIRLACABLE.NS','BIRLACORPN.NS','BSOFT.NS','BBOX.NS','BLISSGVS.NS',
           'BLUEDART.NS','BLUESTARCO.NS','BBTC.NS','BOMDYEING.NS','BRFL.NS','BORORENEW.NS','BOSCHLTD.NS','BRIGADE.NS','BCG.NS',
          'BRITANNIA.NS','CCL.NS','MAPMYINDIA.NS','CESC.NS','CGPOWER.NS','CIEINDIA.NS','CSBBANK.NS','CAMLINFINE.NS',
           'CAMPUS.NS','CANFINHOME.NS','CANBK.NS','CAPLIPOINT.NS','CGCL.NS','CARBORUNIV.NS','CARERATING.NS','CASTROLIND.NS',
          'CEATLTD.NS','CENTRALBK.NS','CDSL.NS','CENTENKA.NS','CENTURYPLY.NS','CENTURYTEX.NS','CERA.NS','CHALET.NS',
           'CHAMBLFERT.NS','CHEMPLASTS.NS','CHENNPETRO.NS','CHOLAHLDNG.NS','CHOLAFIN.NS','CINEVISTA.NS','CIPLA.NS','CUB.NS',
          'CLEAN.NS','COALINDIA.NS','COCHINSHIP.NS','COFFEEDAY.NS','COFORGE.NS','COLPAL.NS','CAMS.NS','CONCORDBIO.NS',
          'CONSOFINVT.NS','CONCOR.NS','COROMANDEL.NS','COSMOFIRST.NS','COX&KINGS.NS','CRAFTSMAN.NS','CREATIVEYE.NS','CREDITACC.NS','CRISIL.NS','CROMPTON.NS','CUMMINSIND.NS',
           'CYBERTECH.NS','CYIENT.NS','DBCORP.NS','DBREALTY.NS','DCBBANK.NS','DCMSHRIRAM.NS','DCW.NS','DICIND.NS',
          'DLF.NS','DABUR.NS','DALBHARAT.NS','DALMIASUG.NS','DATAPATTNS.NS','DEEPAKFERT.NS','DEEPAKNTR.NS','DELHIVERY.NS',
          'DELTACORP.NS','DEN.NS','DEVYANI.NS','DHAMPURSUG.NS','DHANI.NS','DHANBANK.NS','DHANUKA.NS','DBL.NS','DISHTV.NS',
          'DCAL.NS','DIVISLAB.NS','DIXON.NS','DONEAR.NS','LALPATHLAB.NS','DRREDDY.NS','DREDGECORP.NS','DWARKESH.NS',
            'DYNAMATECH.NS','EIDPARRY.NS','EIHOTEL.NS','EPL.NS','EASEMYTRIP.NS','ECLERX.NS','EDELWEISS.NS','EDUCOMP.NS',
           'EICHERMOT.NS','ELECTCAST.NS','ELGIEQUIP.NS','EMAMILTD.NS','ENDURANCE.NS','ENGINERSIN.NS','ENIL.NS','EPIGRAL.NS',
          'EQUITASBNK.NS','ERIS.NS','EROSMEDIA.NS','ESABINDIA.NS','ESCORTS.NS','EVEREADY.NS','EVERESTIND.NS','EKC.NS',
          'EXCELINDUS.NS','EXIDEIND.NS','FDC.NS','NYKAA.NS','FEDERALBNK.NS','FMGOETZE.NS','FACT.NS','FINEORG.NS',
           'FINCABLES.NS','FINPIPE.NS','FSL.NS','FIVESTAR.NS','FLEXITUFF.NS','FORTIS.NS','FOSECOIND.NS','FCONSUMER.NS','FEL.NS',
          'FLFL.NS','FRETAIL.NS','GAIL.NS','GBGLOBAL.NS','GEPIL.NS','GET&D.NS','GFLLIMITED.NS','GHCL.NS','GMMPFAUDLR.NS',
          'GMRINFRA.NS','GRINFRA.NS','GTLINFRA.NS','GTL.NS','GVKPIL.NS','GABRIEL.NS','GALAXYSURF.NS','GAMMONIND6.BO','GRSE.NS',
          'GARFIBRES.NS','ACLGATI.NS','GAYAPROJ.NS','GICRE.NS','GEOJITFSL.NS','GILLETTE.NS','GLAND.NS','GLAXO.NS','GLS.NS',
          'GLENMARK.NS','MEDANTA.NS','GOCOLORS.NS','GPIL.NS','GODFRYPHLP.NS','GODREJAGRO.NS','GODREJCP.NS','GODREJIND.NS',
          'GODREJPROP.NS','GOKEX.NS','GOODYEAR.NS','GRANULES.NS','GRAPHITE.NS','GRASIM.NS','GESHIP.NS','GREAVESCOT.NS',
          'GREENPANEL.NS','GREENPLY.NS','GRINDWELL.NS','GUJALKALI.NS','GAEL.NS','FLUOROCHEM.NS','GUJGASLTD.NS','GIPCL.NS',
          'GMDCLTD.NS','GNFC.NS','GPPL.NS','GSFC.NS','GSPL.NS','GULFOILLUB.NS','HCL-INSYS.NS','HCLTECH.NS','HDFCAMC.NS',
          'HDFCBANK.NS','HDFCLIFE.NS','HEG.NS','HFCL.NS','HLEGLAS.NS','HLVLTD.NS','HMT.NS','HTMEDIA.NS','HAPPSTMNDS.NS',
          'HARRMALAYA.NS','HATHWAY.NS','HATSUN.NS','HAVELLS.NS','HEIDELBERG.NS','HEMIPROP.NS','HERITGFOOD.NS','HEROMOTOCO.NS',
          'HEUBACHIND.NS','HIKAL.NS','HSCL.NS','HIMATSEIDE.NS','HINDALCO.NS','HGS.NS','HAL.NS','HCC.NS','HINDCOPPER.NS','HMVL.NS',
          'HINDMOTORS.NS','HINDOILEXP.NS','HINDPETRO.NS','HINDUNILVR.NS','HINDZINC.NS','POWERINDIA.NS','HOMEFIRST.NS',
          'HONDAPOWER.NS','HONAUT.NS','HUDCO.NS','HDIL.NS','HUBTOWN.NS','HUHTAMAKI.NS','ICICIBANK.NS','ICICIGI.NS',
          'ICICIPRULI.NS','ISEC.NS','ICRA.NS','IDBI.NS','IDFCFIRSTB.NS','IDFC.NS','IFBIND.NS','IFCI.NS','IIFL.NS',
          'IL&FSENGG.NS','IL&FSTRANS.NS','IOLCP.NS','IRB.NS','ITC.NS','ITDCEM.NS','ITI.NS','IGARASHI.NS',]

In [ ]:
tickers = []

In [ ]:
# List to store RSI and SMA values
data_list = []

# Calculate RSI and SMA for each ticker and store in list
for ticker in tickers:
    try:
        rsi_value = calculate_RSI(ticker)
        sma_20_value = calculate_SMA(ticker, window_length=20)
        sma_50_value = calculate_SMA(ticker, window_length=50)
        data_list.append({'Ticker': ticker, 'RSI': float(rsi_value), 'SMA_20': float(sma_20_value), 'SMA_50': float(sma_50_value)})
    except ValueError as e:
        print(f"Error fetching data for {ticker}: {e}")

# Convert list to DataFrame
data_values = pd.DataFrame(data_list)

# Display the RSI and SMA values in a table
print(data_values)

In [74]:
       

# Create a new column based on the function
data_values["Status"] = data_values.apply(Status, axis=1)
data_values

,Ticker,RSI,SMA_20,SMA_50,Status
0,ADCINDIA.BO,73.098706,1365.002496,1237.208005,1.0
1,ALLCARGO.BO,30.774340,65.884001,69.369000,0.0
2,ASHIMASYN.BO,64.809653,33.582000,26.316800,0.0
3,FORBESCO.BO,52.276378,636.209991,618.802998,0.0
4,GTNINDS.BO,55.645162,32.552500,33.005000,0.0
...,...,...,...,...,...
921,ZODIACLOTH.NS,78.961562,113.294500,114.281799,0.0
922,ZOMATO.NS,64.538302,184.263000,188.611200,0.0
923,ZUARIIND.NS,71.498371,310.447499,326.172000,0.0
924,ZYDUSLIFE.NS,67.397537,1061.785007,1016.411007,0.0


In [75]:
buy_list = data_values[data_values['Status'] == 1]['Ticker'].tolist()
sell_list = data_values[data_values['Status'] == -1]['Ticker'].tolist()
hold_list = data_values[data_values['Status'] == 0]['Ticker'].tolist()

print("Buy List:", buy_list)
print("Sell List:", sell_list)
print("Hold List:", hold_list)

Buy List: ['ADCINDIA.BO', 'GLOBOFFS.BO', 'GOODRICKE.BO', 'PUNJCOMMU.BO', '20MICRONS.NS', '3MINDIA.BO', 'AIAENG.NS', 'AAVAS.BO', 'ABSLAMC.NS', 'AKZOINDIA.NS', 'ALKEM.NS', 'ANDHRAPAP.NS', 'ANDHRSUGAR.NS', 'ARVINDFASN.NS', 'ASAHIINDIA.NS', 'ASHOKA.NS', 'ATUL.NS', 'BASF.NS', 'BLKASHYAP.NS', 'INDIAGLYCO.NS', 'BAJAJCON.NS', 'BAJAJELEC.NS', 'BAJAJHIND.NS', 'BALRAMCHIN.NS', 'BANARISUG.NS', 'BAYERCROP.NS', 'BEPL.NS', 'BIKAJI.NS', 'MAPMYINDIA.NS', 'CANFINHOME.NS', 'CERA.NS', 'CHOLAHLDNG.NS', 'CIPLA.NS', 'CLEAN.NS', 'COROMANDEL.NS', 'COSMOFIRST.NS', 'CRAFTSMAN.NS', 'DICIND.NS', 'DELTACORP.NS', 'DHANUKA.NS', 'DIXON.NS', 'DONEAR.NS', 'DWARKESH.NS', 'EIDPARRY.NS', 'ELGIEQUIP.NS', 'ENDURANCE.NS', 'EQUITASBNK.NS', 'ERIS.NS', 'ESCORTS.NS', 'EXIDEIND.NS', 'FMGOETZE.NS', 'FACT.NS', 'FINEORG.NS', 'FIVESTAR.NS', 'FOSECOIND.NS', 'FRETAIL.NS', 'GEPIL.NS', 'GHCL.NS', 'GTLINFRA.NS', 'GABRIEL.NS', 'GALAXYSURF.NS', 'GARFIBRES.NS', 'GPIL.NS', 'GODREJAGRO.NS', 'GRANULES.NS', 'GREENPLY.NS', 'GIPCL.NS', 'HAPPSTMNDS.